In [1]:
# !pip install -q matplotlib

In [2]:
# TODO: Change this as required
CHUNK_NUM = "00"
prefix = "shard_"
input_f = prefix + CHUNK_NUM
output_dir = 'outputs/'

output_f = output_dir  + "output_" + prefix + CHUNK_NUM + '.csv'

In [3]:
import pandas as pd
import re

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

import pickle
from typing import List

import os
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
tqdm.pandas()

In [4]:
df = pd.read_json(input_f, lines=True)

In [5]:
def clean_text(text):
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Remove special characters like &gt;
    text = re.sub(r'&\w+;', '', text)
    return text.lstrip('1234567890')

In [6]:
import time
import pickle

cnt = 0
total = 0

start_time = time.time()

partial_rows = []

for i in range(df.shape[0]):
  for j in range(len(df['posts'][i])):
    total += 1
    if 'perspectives' not in df['posts'][i][j] or 'com' not in df['posts'][i][j] or 'no' not in df['posts'][i][j] or 'now' not in df['posts'][i][j]:
      continue

    ps = df['posts'][i][j]['perspectives']
    c = clean_text(df['posts'][i][j]['com'])
    no = df['posts'][i][j]['no']
    ts = df['posts'][i][j]['now']

    if len(c) < 15:
      continue

    if max(ps.values()) >= 0.5:
      continue

    if 'http' in c:
      continue

    transformed_row = {k: v for k, v in ps.items()}
    transformed_row['com'] = c
    transformed_row['no'] = no
    transformed_row['ts'] = ts

    partial_rows.append(transformed_row)
    cnt += 1
    # print(ps)
    # print(c)
    # print(no)
    # print(ts)

transformed_df = pd.DataFrame(partial_rows)
transformed_df.to_csv(input_f + '_transformed.csv', index_label = 'id')

#pickle.dump(partial_rows, open(input_f + '_transformed.pkl', 'wb'))
end_time = time.time()

print(cnt)
print(total)

secs = end_time - start_time
per_entry = secs / total
print(f"For 100k entries it should take: {100000 * per_entry}")
print(f"Filtered out {100.0 * (total - cnt) / total} % of entries")


70921
250756
For 100k entries it should take: 7.0122561267427495
Filtered out 71.71712740672207 % of entries


In [7]:
final_df = pd.read_csv(input_f + '_transformed.csv')
final_df

,id,TOXICITY,SEVERE_TOXICITY,INFLAMMATORY,PROFANITY,INSULT,OBSCENE,SPAM,com,no,ts
0,0,0.281835,0.152321,0.306632,0.225176,0.194891,0.146112,0.103734,what thread are you trying to slide?saged and ...,80566489,07/11/16(Mon)00:23:10
1,1,0.036423,0.014113,0.154549,0.015209,0.017134,0.044861,0.372402,all those green texts,80567010,07/11/16(Mon)00:29:33
2,2,0.202302,0.141011,0.185279,0.147017,0.220031,0.179816,0.397897,yfw someone asks you to read something,80567361,07/11/16(Mon)00:33:43
3,3,0.057361,0.032460,0.100461,0.035854,0.050958,0.034121,0.079154,god bless you officer bob,80559043,07/10/16(Sun)22:58:13
4,4,0.093544,0.027282,0.194140,0.033801,0.064638,0.017152,0.025519,when does the US presidential race heat up or ...,80542072,07/10/16(Sun)19:55:01
...,...,...,...,...,...,...,...,...,...,...,...
70916,70916,0.377718,0.312394,0.472712,0.214903,0.320847,0.156911,0.162956,youll never hug her and give her children,80948923,07/14/16(Thu)06:23:30
70917,70917,0.307341,0.210054,0.150781,0.158933,0.270499,0.069753,0.458293,Turkeynot buying DAESH oiltry again,80950293,07/14/16(Thu)06:49:32
70918,70918,0.143771,0.109660,0.278799,0.107041,0.126884,0.315545,0.427122,When will Mısır be more developed?,80951080,07/14/16(Thu)07:04:14
70919,70919,0.277167,0.095080,0.324296,0.125312,0.231076,0.037293,0.102966,I know this is ironicBut Israel is much more f...,80952112,07/14/16(Thu)07:20:37


In [8]:
# Load the model and tokenizer once
model_name = 'microsoft/DialoGPT-medium'
tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer.padding_side = "left"
# tokenizer.pad_token = '<pad>'
model = AutoModelForCausalLM.from_pretrained(model_name).to('cuda')

def get_dialogpt_response(comment):
    # Tokenize the input comment
    input_ids = tokenizer.encode(comment + tokenizer.eos_token, return_tensors="pt").to('cuda')

    # Generate a response
    response_ids = model.generate(input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(response_ids[:, input_ids.shape[-1]:][0], skip_special_tokens=True)

    return response

In [9]:
import pandas as pd

def process_and_save(final_df, func, b_chunk_size=200):
    """Applies a function to the DataFrame in chunks and saves periodically."""

    for i in range(0, len(final_df), b_chunk_size):
        chunk = final_df.iloc[i:i + b_chunk_size]
        chunk['response'] = chunk['com'].progress_apply(func)

        # Append to CSV file (mode='a' for append)
        chunk.to_csv(output_f, mode='a', header=i == 0, index=False)

process_and_save(final_df, get_dialogpt_response)

  0%|          | 0/200 [00:00<?, ?it/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/scratch/1028977/ipykernel_2159504/774280913.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['response'] = chunk['com'].progress_apply(func)


  0%|          | 0/200 [00:00<?, ?it/s]

/scratch/1028977/ipykernel_2159504/774280913.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['response'] = chunk['com'].progress_apply(func)


  0%|          | 0/200 [00:00<?, ?it/s]

...

/scratch/1028977/ipykernel_2159504/774280913.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['response'] = chunk['com'].progress_apply(func)


  0%|          | 0/200 [00:00<?, ?it/s]

/scratch/1028977/ipykernel_2159504/774280913.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['response'] = chunk['com'].progress_apply(func)


  0%|          | 0/200 [00:00<?, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [10]:
sanity = pd.read_csv(output_f)
sanity

,id,TOXICITY,SEVERE_TOXICITY,INFLAMMATORY,PROFANITY,INSULT,OBSCENE,SPAM,com,no,ts,response
0,0,0.281835,0.152321,0.306632,0.225176,0.194891,0.146112,0.103734,what thread are you trying to slide?saged and ...,80566489,07/11/16(Mon)00:23:10,I'm trying to slide the thread.
1,1,0.036423,0.014113,0.154549,0.015209,0.017134,0.044861,0.372402,all those green texts,80567010,07/11/16(Mon)00:29:33,"I'm not sure if you're being sarcastic or not,..."
2,2,0.202302,0.141011,0.185279,0.147017,0.220031,0.179816,0.397897,yfw someone asks you to read something,80567361,07/11/16(Mon)00:33:43,I'm not sure what you mean by that.
3,3,0.057361,0.032460,0.100461,0.035854,0.050958,0.034121,0.079154,god bless you officer bob,80559043,07/10/16(Sun)22:58:13,"I'm not an officer, I'm a civilian."
4,4,0.093544,0.027282,0.194140,0.033801,0.064638,0.017152,0.025519,when does the US presidential race heat up or ...,80542072,07/10/16(Sun)19:55:01,I think it's a toss up between her and Bernie.
...,...,...,...,...,...,...,...,...,...,...,...,...
70916,70916,0.377718,0.312394,0.472712,0.214903,0.320847,0.156911,0.162956,youll never hug her and give her children,80948923,07/14/16(Thu)06:23:30,"I'm not sure if you're being serious or not, b..."
70917,70917,0.307341,0.210054,0.150781,0.158933,0.270499,0.069753,0.458293,Turkeynot buying DAESH oiltry again,80950293,07/14/16(Thu)06:49:32,I'm sure they will.
70918,70918,0.143771,0.109660,0.278799,0.107041,0.126884,0.315545,0.427122,When will Mısır be more developed?,80951080,07/14/16(Thu)07:04:14,When will the Turkish government be more devel...
70919,70919,0.277167,0.095080,0.324296,0.125312,0.231076,0.037293,0.102966,I know this is ironicBut Israel is much more f...,80952112,07/14/16(Thu)07:20:37,I'm not sure what you're trying to say.
